## Breast_Cancer_Prediction_Using_NN_Module

In [17]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [18]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [19]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace= True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [20]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,2:],df.iloc[:,0], train_size= 0.2)


In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [23]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [24]:
X_train.dtype

torch.float32

In [25]:
learning_parameter = 0.1
epochs = 1000

## Feed Forward NN with Two Hidden Layer

## Using Sequential Container

In [26]:
class model_seq(nn.Module):
    def __init__(self,X_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(X_size,10),
            nn.ReLU(),
            nn.Linear(10,5),
            nn.ReLU(),
            nn.Linear(5,1),
            nn.Sigmoid()
        )

    def forward(self, X_data):
        output = self.network(X_data)
        return output


In [27]:
cancer_model = model_seq(X_train.shape[1])

In [28]:
loss_function = nn.BCELoss()
optimizer = torch.optim.SGD(cancer_model.parameters(),lr=learning_parameter)

## Creating Custom Dataset 

In [29]:
from torch.utils.data import Dataset, DataLoader
class customDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.target = y

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self,index):
        return self.features[index],self.target[index]

In [31]:
train_dataset = customDataset(X_train,y_train)
test_dataset = customDataset(X_test,y_test)

In [32]:
train_dataloader = DataLoader(train_dataset, batch_size= 10, shuffle = False)
test_dataloader = DataLoader(test_dataset,batch_size = 10, shuffle = False)

In [33]:
for epoch in range(epochs):
    for X_batch, y_batch in train_dataloader:
        y_pred = cancer_model(X_batch)
        loss = loss_function(y_pred, y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1} is {loss}')

Epoch 1 is 0.6803104281425476
Epoch 1 is 0.6725075244903564
Epoch 1 is 0.6848042011260986
Epoch 1 is 0.6055277585983276
Epoch 1 is 0.7256760001182556
Epoch 1 is 0.721591055393219
Epoch 1 is 0.7085920572280884
Epoch 1 is 0.6289058923721313
Epoch 1 is 0.6526325941085815
Epoch 1 is 0.6225544214248657
Epoch 1 is 0.6100476980209351
Epoch 1 is 0.5368879437446594
Epoch 2 is 0.6100708246231079
Epoch 2 is 0.6359068751335144
Epoch 2 is 0.6050957441329956
Epoch 2 is 0.5192807912826538
Epoch 2 is 0.6424812078475952
Epoch 2 is 0.6434226632118225
Epoch 2 is 0.5991202592849731
Epoch 2 is 0.6040142178535461
Epoch 2 is 0.5235726237297058
Epoch 2 is 0.5472768545150757
Epoch 2 is 0.4626918435096741
Epoch 2 is 0.4521699845790863
Epoch 3 is 0.49450749158859253
Epoch 3 is 0.5782299041748047
Epoch 3 is 0.46489280462265015
Epoch 3 is 0.4269079267978668
Epoch 3 is 0.4763730466365814
Epoch 3 is 0.47714725136756897
Epoch 3 is 0.4049977660179138
Epoch 3 is 0.5671265125274658
Epoch 3 is 0.35610365867614746
Epoch 3

In [35]:
accuracy = []

In [36]:
with torch.no_grad():
    for X_batch, y_batch in test_dataloader:
        y_pred = cancer_model(X_batch)
        y_pred = (y_pred > 0.6).float()
        accu = (y_pred == y_batch).float().mean().item()
        accuracy.append(accu)


In [37]:
sum(accuracy)/len(accuracy)

0.5943478229253188